In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ffipy import FFIEC_Client
from io import StringIO
import time

In [2]:
client = FFIEC_Client()
# get a list of RSSD codes for filers on 9/30/2010
filers_RSSD_09302010 = client.retrieve_filers_since_date(ds_name='Call',
                                   reporting_pd_end='9/30/2010',
                                   last_update_date='9/30/2010')
# check how many filers - 5908
len(filers_RSSD_09302010)

7094

In [3]:
# Testing Call report for 1 filer
ds_name = 'Call'    # Pull Call report data
end_date = '9/30/2010'  # Pull data in reporting pd ending 3/31/17
fiID_type = 'ID_RSSD'  # Type of financial inst identifier
fiID = 688556    # Indentifier for Wyomin Bank and Trust
fmt = 'SDF'  # Pull report as PDF
# <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
outfile = None  # Output file path
return_result = True  # If True, method returns the data
results_bytes = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                              fiID_type=fiID_type, fiID=fiID,
                              facsimile_fmt=fmt, outfile=None,
                              return_result=True)
report = StringIO(str(results_bytes, 'utf-8'))
report_df = pd.read_csv(report, sep=';')



In [5]:
report_df.head(10)

,Call Date,Bank RSSD Identifier,MDRM #,Value,Last Update,Short Definition,Call Schedule,Line Number
0,20100930,688556,RCON0010,14201,20101012,Cash and balances dues from depository institu...,RCR,34
1,20100930,688556,RCON0020,0,20101012,Cash items in process of collection and unpost...,RCA,1a
2,20100930,688556,RCON0071,12625,20101012,Interest-bearing balances,RC,1b
3,20100930,688556,RCON0073,0,20101012,Foreign branches of other U.S. banks,RCA,3a
4,20100930,688556,RCON0074,0,20101012,Other banks in foreign countries and foreign c...,RCA,3b
5,20100930,688556,RCON0080,1006,20101012,Currency and coin,RCA,1b
6,20100930,688556,RCON0081,1577,20101012,Noninterest-bearing balances and currency and ...,RC,1a
7,20100930,688556,RCON0083,0,20101012,U.S. branches and agencies of foreign banks,RCA,2a
8,20100930,688556,RCON0085,13097,20101012,Other commercial banks in the U.S. and other d...,RCA,2b
9,20100930,688556,RCON0090,98,20101012,Balances due from Federal Reserve Banks,RCA,4


In [3]:
# Prepare dictionary of selected financials that are needed for CAMELS
selected_financials = {
                       'RIAD3210' : 'Total equity capital', # Alt can be RCONG105
                       'RCON2170' : 'Total assets', # Ok
                       'RCON3360' : 'Total loans', #ok
                       'RCON3465' : '1-4 family residential loans', 
                       'RCON3466' : 'Other real estate loans',
                       'RCON3387' : 'Commercial and industrial loans',
                       'RCONB561' : 'Credit cards',
                       'RCON3123' : 'Allowance for loan losses', # OK
                       'RIAD4093' : 'Total noninterest expense', #OK
                       'RIAD4300' : 'Net Income before', # # new for healthy
                       'RCON2215' : 'Total transaction deposits', # ok
                       'RCON2385' : 'Total nontransaction deposits', # ok
                       'RCON1773' : 'Available-for-sale Fair Value',
                       'RIAD4150' : 'Number of full-time employees'# new for healthy
                      } 


In [6]:
# Expected 14 columns, store column names in the list columns
len(selected_financials.keys())
columns = list(selected_financials.keys())
len(columns)

14

In [153]:
# Pull 7000+ reports for filers_RSSD_09302010, drop all values but 13 selected metrics
all_data_df = pd.DataFrame()
#fin_data_dic = {}
report_data_dict = {}

for i in range(len(filers_RSSD_09302010)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '9/30/2010'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'ID_RSSD'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = filers_RSSD_09302010[i]   # Indentifier for Wyomin Bank and Trust
    #print("counter at ", i, " pulling RSSD# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        key = filers_RSSD_09302010[i]
        report_data_dict[key] = {}
        for col in columns:
            df = report_df.loc[report_df['MDRM #'] == col]
            value = df.Value.values[0]
            report_data_dict[key][col] = value
        #report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       #'Short Definition', 'Call Schedule', 'Line Number']
       # values = list(report_df[report_df.ValueID.isin(columns)].Value)
       # fin_data_dic[fiID] = values
        #all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

In [165]:
# Inspect the data pulled

healthy_after_2010Q3 = pd.DataFrame.from_dict(report_data_dict, orient='index')
#len(healthy_after_2010Q3.index)
#healthy_after_2010Q3.describe()
#healthy_after_2010Q3.columns
#store the data
#healthy_after_2010Q3.to_csv('data/healthy_after_2010Q3')


In [173]:
healthy_after_2010Q3.columns = selected_financials.values()
#store the data
#healthy_after_2010Q3.to_csv('data/healthy_after_2010Q3')


In [56]:
filers_RSSD_09302010.index(3669930) # expected 3669930
len(filers_RSSD_09302010)

7094

In [ ]:
filers = filers_RSSD_09302010
#all_batch_data_df = pd.DataFrame()
report_data_dict = {}
batches_to_pull = np.array_split(filers,100)

for n in range(len(batches_to_pull)):
    for i in range(len(batches_to_pull[n])):
        ds_name = 'Call'    # Pull Call report data
        end_date = '6/30/2010'  # Pull data in reporting pd ending 3/31/17
        fiID_type = 'ID_RSSD'  # Type of financial inst identifier (ID_RSSD, CERT )
        fiID = batches_to_pull[n][i]   # Indentifier for Wyomin Bank and Trust
        #print("counter at ", i, " pulling RSSD# ", fiID)
        fmt = 'SDF'  # Pull report as PDF
        return_result = True  # If True, method returns the data
        
        try:
            f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                          fiID_type=fiID_type, fiID=fiID,
                                          facsimile_fmt=fmt, outfile=False,
                                          return_result=True)
            report = StringIO(str(f, 'utf-8'))
            report_df = pd.read_csv(report, sep=';')
            key = fiID #filers_RSSD_09302010[i]    # <---- changed to fiID
            report_data_dict[key] = {}

            for col in columns:
                df = report_df.loc[report_df['MDRM #'] == col]
                value = df.Value.values[0]
                report_data_dict[key][col] = value
            
            batch_data_df = pd.DataFrame.from_dict(report_data_dict, orient='index') 


        except:
            next 
    
    print('Loading batch number ', n)
    batch_data_df = pd.DataFrame.from_dict(report_data_dict, orient='index')  
    #all_batch_data_df = all_batch_data_df.append(batch_data_df)#, ignore_index=True)
   

Loading batch number  0
Loading batch number  1
Loading batch number  2
Loading batch number  3
Loading batch number  4
Loading batch number  5
Loading batch number  6
Loading batch number  7
Loading batch number  8
Loading batch number  9
Loading batch number  10
Loading batch number  11
Loading batch number  12
Loading batch number  13


In [61]:
batch_data_df.shape

(20, 14)

In [25]:
batches_to_pull = np.array_split(filers,5)
for n in range(len(batches_to_pull)):
    for i in range(len(batches_to_pull[n])):
        print(batches_to_pull[n][i])

175458
660066
856159
1001451
723158
522669
431846
854351
178057
652753
835743
980438
1216992
371849
913641
734949
97943
365875
24659
898850


In [ ]:
# Pull 7000+ reports for filers_RSSD_09302010 for Q-1, 06302010, drop all values but 13 selected metrics -- returns only 27
batch_data_df = pd.DataFrame()
#fin_data_dic = {}
report_data_dict = {}

for i in range(len(filers_RSSD_09302010)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '6/30/2010'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'ID_RSSD'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = filers_RSSD_09302010[i]   # Indentifier for Wyomin Bank and Trust
    print("counter at ", i, " pulling RSSD# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        key = filers_RSSD_09302010[i]
        report_data_dict[key] = {}
        for col in columns:
            df = report_df.loc[report_df['MDRM #'] == col]
            value = df.Value.values[0]
            report_data_dict[key][col] = value
        #report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       #'Short Definition', 'Call Schedule', 'Line Number']
       # values = list(report_df[report_df.ValueID.isin(columns)].Value)
       # fin_data_dic[fiID] = values
        #all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

counter at  0  pulling RSSD#  175458
counter at  1  pulling RSSD#  660066
counter at  2  pulling RSSD#  856159
counter at  3  pulling RSSD#  1001451
counter at  4  pulling RSSD#  723158
counter at  5  pulling RSSD#  522669
counter at  6  pulling RSSD#  431846
counter at  7  pulling RSSD#  854351
counter at  8  pulling RSSD#  178057
counter at  9  pulling RSSD#  652753
counter at  10  pulling RSSD#  835743
counter at  11  pulling RSSD#  980438
counter at  12  pulling RSSD#  1216992
counter at  13  pulling RSSD#  371849
counter at  14  pulling RSSD#  913641
counter at  15  pulling RSSD#  734949
counter at  16  pulling RSSD#  97943
counter at  17  pulling RSSD#  365875
counter at  18  pulling RSSD#  24659
counter at  19  pulling RSSD#  898850
counter at  20  pulling RSSD#  630340
counter at  21  pulling RSSD#  910659
counter at  22  pulling RSSD#  595038
counter at  23  pulling RSSD#  116732
counter at  24  pulling RSSD#  489249
counter at  25  pulling RSSD#  386357
counter at  26  pullin

counter at  215  pulling RSSD#  504647
counter at  216  pulling RSSD#  51253
counter at  217  pulling RSSD#  376068
counter at  218  pulling RSSD#  3649624
counter at  219  pulling RSSD#  272151
counter at  220  pulling RSSD#  111559
counter at  221  pulling RSSD#  685733
counter at  222  pulling RSSD#  340443
counter at  223  pulling RSSD#  563457
counter at  224  pulling RSSD#  121651
counter at  225  pulling RSSD#  141350
counter at  226  pulling RSSD#  3571015
counter at  227  pulling RSSD#  242949
counter at  228  pulling RSSD#  85454
counter at  229  pulling RSSD#  641252
counter at  230  pulling RSSD#  601032
counter at  231  pulling RSSD#  472045
counter at  232  pulling RSSD#  856542
counter at  233  pulling RSSD#  976347
counter at  234  pulling RSSD#  552956
counter at  235  pulling RSSD#  825847
counter at  236  pulling RSSD#  426141
counter at  237  pulling RSSD#  715032
counter at  238  pulling RSSD#  924058
counter at  239  pulling RSSD#  996251
counter at  240  pulling 

counter at  425  pulling RSSD#  1016941
counter at  426  pulling RSSD#  411549
counter at  427  pulling RSSD#  695947
counter at  428  pulling RSSD#  138725
counter at  429  pulling RSSD#  999355
counter at  430  pulling RSSD#  548267
counter at  431  pulling RSSD#  726355
counter at  432  pulling RSSD#  721752
counter at  433  pulling RSSD#  257345
counter at  434  pulling RSSD#  418100
counter at  435  pulling RSSD#  57255
counter at  436  pulling RSSD#  278733
counter at  437  pulling RSSD#  823638
counter at  438  pulling RSSD#  380346
counter at  439  pulling RSSD#  1011432
counter at  440  pulling RSSD#  797319
counter at  441  pulling RSSD#  116639
counter at  442  pulling RSSD#  188926
counter at  443  pulling RSSD#  3595785
counter at  444  pulling RSSD#  2362074
counter at  445  pulling RSSD#  239042
counter at  446  pulling RSSD#  997847
counter at  447  pulling RSSD#  613754
counter at  448  pulling RSSD#  269049
counter at  449  pulling RSSD#  335757
counter at  450  pulli

counter at  635  pulling RSSD#  3449730
counter at  636  pulling RSSD#  498942
counter at  637  pulling RSSD#  948643
counter at  638  pulling RSSD#  612159
counter at  639  pulling RSSD#  912578
counter at  640  pulling RSSD#  941653
counter at  641  pulling RSSD#  74056
counter at  642  pulling RSSD#  274052
counter at  643  pulling RSSD#  126553
counter at  644  pulling RSSD#  803658
counter at  645  pulling RSSD#  3269590
counter at  646  pulling RSSD#  622756
counter at  647  pulling RSSD#  160557
counter at  648  pulling RSSD#  296559
counter at  649  pulling RSSD#  3145162
counter at  650  pulling RSSD#  403656
counter at  651  pulling RSSD#  890957
counter at  652  pulling RSSD#  771047
counter at  653  pulling RSSD#  799939
counter at  654  pulling RSSD#  132853
counter at  655  pulling RSSD#  705453
counter at  656  pulling RSSD#  923949
counter at  657  pulling RSSD#  641449
counter at  658  pulling RSSD#  887854
counter at  659  pulling RSSD#  3448180
counter at  660  pulli

counter at  846  pulling RSSD#  908553
counter at  847  pulling RSSD#  226949
counter at  848  pulling RSSD#  119153
counter at  849  pulling RSSD#  634656
counter at  850  pulling RSSD#  869346
counter at  851  pulling RSSD#  857745
counter at  852  pulling RSSD#  719610
counter at  853  pulling RSSD#  890050
counter at  854  pulling RSSD#  304155
counter at  855  pulling RSSD#  2746245
counter at  856  pulling RSSD#  649342
counter at  857  pulling RSSD#  729011
counter at  858  pulling RSSD#  3822445
counter at  859  pulling RSSD#  706030
counter at  860  pulling RSSD#  2823221
counter at  861  pulling RSSD#  372640
counter at  862  pulling RSSD#  479053
counter at  863  pulling RSSD#  335355
counter at  864  pulling RSSD#  551016
counter at  865  pulling RSSD#  837046
counter at  866  pulling RSSD#  3718548
counter at  867  pulling RSSD#  521064
counter at  868  pulling RSSD#  646958
counter at  869  pulling RSSD#  718556
counter at  870  pulling RSSD#  297547
counter at  871  pull

...

counter at  5577  pulling RSSD#  947628
counter at  5578  pulling RSSD#  3468450
counter at  5579  pulling RSSD#  498456
counter at  5580  pulling RSSD#  1917301
counter at  5581  pulling RSSD#  668848
counter at  5582  pulling RSSD#  378044
counter at  5583  pulling RSSD#  199454
counter at  5584  pulling RSSD#  3350658
counter at  5585  pulling RSSD#  2785477
counter at  5586  pulling RSSD#  3335301
counter at  5587  pulling RSSD#  960047
counter at  5588  pulling RSSD#  2358769
counter at  5589  pulling RSSD#  631039
counter at  5590  pulling RSSD#  2941509
counter at  5591  pulling RSSD#  398051
counter at  5592  pulling RSSD#  4156
counter at  5593  pulling RSSD#  216445
counter at  5594  pulling RSSD#  57040
counter at  5595  pulling RSSD#  3564286
counter at  5596  pulling RSSD#  944355
counter at  5597  pulling RSSD#  262844
counter at  5598  pulling RSSD#  2718596
counter at  5599  pulling RSSD#  2860440
counter at  5600  pulling RSSD#  25357
counter at  5601  pulling RSSD#  8

counter at  5781  pulling RSSD#  2587916
counter at  5782  pulling RSSD#  435750
counter at  5783  pulling RSSD#  1005552
counter at  5784  pulling RSSD#  2468097
counter at  5785  pulling RSSD#  2910121
counter at  5786  pulling RSSD#  463735
counter at  5787  pulling RSSD#  667757
counter at  5788  pulling RSSD#  3613652
counter at  5789  pulling RSSD#  495857
counter at  5790  pulling RSSD#  3020447
counter at  5791  pulling RSSD#  75053
counter at  5792  pulling RSSD#  586027
counter at  5793  pulling RSSD#  960935
counter at  5794  pulling RSSD#  165039
counter at  5795  pulling RSSD#  697231
counter at  5796  pulling RSSD#  147633
counter at  5797  pulling RSSD#  260055
counter at  5798  pulling RSSD#  534774
counter at  5799  pulling RSSD#  794608
counter at  5800  pulling RSSD#  160959
counter at  5801  pulling RSSD#  253730
counter at  5802  pulling RSSD#  705556
counter at  5803  pulling RSSD#  87234
counter at  5804  pulling RSSD#  548351
counter at  5805  pulling RSSD#  883

counter at  5986  pulling RSSD#  371362
counter at  5987  pulling RSSD#  321152
counter at  5988  pulling RSSD#  655651
counter at  5989  pulling RSSD#  3076284
counter at  5990  pulling RSSD#  3057818
counter at  5991  pulling RSSD#  209362
counter at  5992  pulling RSSD#  19936
counter at  5993  pulling RSSD#  1008674
counter at  5994  pulling RSSD#  591320
counter at  5995  pulling RSSD#  2785637
counter at  5996  pulling RSSD#  618807
counter at  5997  pulling RSSD#  644440
counter at  5998  pulling RSSD#  764151
counter at  5999  pulling RSSD#  239275
counter at  6000  pulling RSSD#  3278547
counter at  6001  pulling RSSD#  3389582
counter at  6002  pulling RSSD#  509857
counter at  6003  pulling RSSD#  2360904
counter at  6004  pulling RSSD#  3284397
counter at  6005  pulling RSSD#  50461
counter at  6006  pulling RSSD#  767255
counter at  6007  pulling RSSD#  388940
counter at  6008  pulling RSSD#  3016347
counter at  6009  pulling RSSD#  3269844
counter at  6010  pulling RSSD# 

counter at  6190  pulling RSSD#  375043
counter at  6191  pulling RSSD#  663955
counter at  6192  pulling RSSD#  76153
counter at  6193  pulling RSSD#  187648
counter at  6194  pulling RSSD#  425957
counter at  6195  pulling RSSD#  544335
counter at  6196  pulling RSSD#  92706
counter at  6197  pulling RSSD#  2643595
counter at  6198  pulling RSSD#  485214
counter at  6199  pulling RSSD#  3032833
counter at  6200  pulling RSSD#  172457
counter at  6201  pulling RSSD#  118736
counter at  6202  pulling RSSD#  1012671
counter at  6203  pulling RSSD#  564155
counter at  6204  pulling RSSD#  371223
counter at  6205  pulling RSSD#  849777
counter at  6206  pulling RSSD#  479071
counter at  6207  pulling RSSD#  3599055
counter at  6208  pulling RSSD#  641047
counter at  6209  pulling RSSD#  514655
counter at  6210  pulling RSSD#  393252
counter at  6211  pulling RSSD#  434539
counter at  6212  pulling RSSD#  718079
counter at  6213  pulling RSSD#  899736
counter at  6214  pulling RSSD#  10120

counter at  6394  pulling RSSD#  3415463
counter at  6395  pulling RSSD#  816603
counter at  6396  pulling RSSD#  595430
counter at  6397  pulling RSSD#  698957
counter at  6398  pulling RSSD#  293437
counter at  6399  pulling RSSD#  163549
counter at  6400  pulling RSSD#  382050
counter at  6401  pulling RSSD#  717924
counter at  6402  pulling RSSD#  1008553
counter at  6403  pulling RSSD#  2037185
counter at  6404  pulling RSSD#  3393785
counter at  6405  pulling RSSD#  579636
counter at  6406  pulling RSSD#  947655
counter at  6407  pulling RSSD#  3557895
counter at  6408  pulling RSSD#  2614344
counter at  6409  pulling RSSD#  209148
counter at  6410  pulling RSSD#  956956
counter at  6411  pulling RSSD#  917331
counter at  6412  pulling RSSD#  42055
counter at  6413  pulling RSSD#  2872845
counter at  6414  pulling RSSD#  302665
counter at  6415  pulling RSSD#  2371816
counter at  6416  pulling RSSD#  3487518
counter at  6417  pulling RSSD#  1005440
counter at  6418  pulling RSSD#

In [ ]:
# Inspect the data pulled

healthy_after_2010Q2 = pd.DataFrame.from_dict(report_data_dict, orient='index')
#len(healthy_after_2010Q2.index)
#healthy_after_2010Q2.describe()
#healthy_after_2010Q2.columns

#store the data
#healthy_after_2010Q2.to_csv('data/healthy_after_2010Q2')

In [ ]:

healthy_after_2010Q2.columns = selected_financials.values()

#store the data
#healthy_after_2010Q2.to_csv('data/healthy_after_2010Q2')